#### Тема "Part-of-Speech разметка, NER , извлечение отношений."

##### Подготовка:

In [1]:
import pandas as pd
import numpy as np
import nltk

import spacy
import en_core_web_md

In [2]:
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

In [3]:
processed_df = pd.read_pickle('tweet_data.pkl')
processed_df.head(2)

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."


##### Задание 1. 
Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете.   
Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)

С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?
Повторим шаги из заданий 1 и 2, используя библиотеку nltk

In [4]:
nlp = en_core_web_md.load()

In [5]:
processed_df['NER_spacy'] = processed_df['clean_tweet'].apply(lambda x:  nlp(x))

In [6]:
processed_df[['NER_spacy']].head(10)

,NER_spacy
0,"(when, father, is, dysfunctional, and, is, so,..."
1,"(thanks, for, lyft, credit, can, not, use, cau..."
2,"(bihday, your, majesty)"
3,"(model, love, you, take, with, you, all, the, ..."
4,"(factsguide, society, now, motivation)"
5,"(huge, fan, fare, and, big, talking, before, t..."
6,"(camping, tomorrow, danny)"
7,"(the, next, school, year, is, the, year, for, ..."
8,"(we, won, love, the, land, allin, cavs, champi..."
9,"(welcome, here, am, it, has, it, is, so, gr)"


In [7]:
def ners_entities(ner_list):
    ners = []
    for doc in ner_list:
        for ent in doc.ents:
            ners.append((ent.text, ent.label_))

    return pd.DataFrame(ners, columns=['word', 'NER'])

In [8]:
df_ner = ners_entities(processed_df['NER_spacy'].tolist())
df_ner.head()

,word,NER
0,pdx,GPE
1,getthanked,PERSON
2,bihday,DATE
3,tomorrow,DATE
4,danny,PERSON


Самые популярыне типы NER:

In [9]:
df_ner['NER'].value_counts().head(20)

PERSON         11562
DATE           10821
ORG             8108
GPE             5415
TIME            2302
NORP            1701
CARDINAL        1079
ORDINAL          585
FAC              371
PRODUCT          326
LOC              284
EVENT            220
WORK_OF_ART      106
QUANTITY          55
LANGUAGE          40
MONEY             20
LAW               11
PERCENT            4
Name: NER, dtype: int64

In [10]:
df_ner[df_ner['NER']=='PERSON']['word'].value_counts().head(20)

bing bong bing bong                                        107
suppo                                                       69
obama                                                       67
hillary                                                     55
sta                                                         45
melancholy melancholymusic                                  40
feminismiscancer feminismisterrorism feminismmuktbharat     40
bjp                                                         36
christina grimmie                                           34
hu                                                          30
jo cox                                                      29
lebron                                                      26
tgif ff                                                     26
sea shepherd suppoers                                       25
gamedev indiedev indiegamedev                               25
donald trump                                           

In [11]:
df_ner[df_ner['NER']=='ORG']['word'].value_counts().head(20)

sjw                 102
allahsoil            84
gop                  76
nba                  59
islam                53
trump                50
ht                   42
facebook             40
cavs                 39
bihday               39
social analytics     34
cnn                  30
ios                  30
obama                30
google               29
wso                  28
eu                   26
congress             22
kkk                  20
spo                  20
Name: word, dtype: int64

#### Задание 2.  
Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. 
Обратите внимание, что nltk чувствителен к регистру.
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

In [12]:
text = processed_df['clean_tweet'].str.upper().to_string()

In [13]:
chunks = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))) 

In [14]:
ners = {
            (' '.join(c[0] for c in chunk), chunk.label() ) 
            for chunk in chunks 
            if hasattr(chunk, 'label') 
        }

In [15]:
df_ner = pd.DataFrame(ners, columns=['word', 'NER'])
df_ner.head()

,word,NER
0,CLEVELAND,ORGANIZATION
1,LOL,ORGANIZATION
2,PENGUINS,ORGANIZATION
3,CONGRESSMAN,ORGANIZATION
4,MITWAMOL,ORGANIZATION


In [16]:
df_ner['NER'].value_counts().head(20)

ORGANIZATION    12570
PERSON             93
GPE                26
FACILITY           20
GSP                 1
Name: NER, dtype: int64

In [17]:
df_ner[df_ner['NER']=='PERSON']['word'].value_counts().head(20)

DEATH AN           1
ALBUM              1
JOHNSON            1
OBAMA WAS          1
LESSON ON          1
DAVID REMICK       1
V                  1
STEPHEN KESHI      1
BARRY PROBABLY     1
CARES ABO          1
ARABIC             1
DAVID PROPER       1
OBAMA              1
CLICK RAMADAN      1
LESSON PLAN FOR    1
ARABIAN SPICED     1
CARRY GUN          1
CLINTON ARE        1
JOHN BURR          1
STEADY SUB         1
Name: word, dtype: int64

In [18]:
df_ner[df_ner['NER']=='ORGANIZATION']['word'].value_counts().head(20)

PROGRAMMELEADER    1
HUFFINGTON         1
TRENEREASTA        1
BILLCOSBY          1
FLIGHT             1
JEAN               1
VOLTRON            1
YEH                1
AIM                1
BEAUTIES           1
FALL               1
TRIBEON            1
HABITS             1
UNIVERSE           1
MULTIGRAIN         1
COMEBACK           1
SHOPNOW            1
THANKYOUSTUDIO     1
HEA                1
WALLS OF           1
Name: word, dtype: int64